<img src='task.png' width='500px'>

In [1]:
! python3 -m pip install torch
! python3 -m pip install torchvision
! python3 -m pip install scikit-image

In [2]:
import os
import numpy as np
import shutil
from PIL import Image
from matplotlib.pyplot import imshow
from IPython.display import display
%matplotlib inline

<h1>Data Processing</h1>

In [3]:
categories = ('male', 'female')
root_dir = 'data'

In [4]:
test_ratio = 0.1

for category in categories:
    shutil.rmtree(root_dir +'/train/' + category)
    os.makedirs(root_dir +'/train/' + category)
    shutil.rmtree(root_dir +'/test/' + category)
    os.makedirs(root_dir +'/test/' + category)
    
    copy_from_dir = 'photos/' + category
    
    all_filenames = os.listdir(copy_from_dir)
    np.random.shuffle(all_filenames)
    train_filenames, test_filenames = np.split(
        np.array(all_filenames),
        [int(len(all_filenames) * (1 - test_ratio))]
    )
    train_filenames = [copy_from_dir + '/' + name for name in train_filenames.tolist()]
    test_filenames = [copy_from_dir + '/' + name for name in test_filenames.tolist()]
    
    print(f'Category {category}.')
    print('Number of total images: ', len(all_filenames))
    print('Training: ', len(train_filenames))
    print('Testing: ', len(test_filenames))
    
    for name in train_filenames:
        shutil.copy(name, root_dir + '/train/' + category)

    for name in test_filenames:
        shutil.copy(name, root_dir +'/test/' + category)

FileNotFoundError: [Errno 2] No such file or directory: 'data/train/male'

to learn images ratio:

In [5]:
max_ratio = 0
min_width = 1000000000
max_width = 0
min_height = 100000000
max_height = 0

for category in categories:
    source_dir = 'photos/' + category
    
    for photo in os.listdir(source_dir):
        im = Image.open(source_dir + '/' + photo)
        width, height = im.size
        if min_width > width:
            min_width = width
        if max_width < width:
            max_width = width
        if min_height > height:
            min_height = height
        if max_height < height:
            max_height = height
        if max(width, height) / min(width, height) > max_ratio:
            max_ratio = max(width, height) / min(width, height)

print(f'min width: {min_width}')
print(f'max width: {max_width}')
print(f'min height: {min_height}')
print(f'max height: {max_height}')
print(f'max ratio: {max_ratio}')

FileNotFoundError: [Errno 2] No such file or directory: 'photos/male'

In [6]:
cnt_small = 0

for category in categories:
    source_dir = 'photos/' + category
    
    for photo in os.listdir(source_dir):
        im = Image.open(source_dir + '/' + photo)
        width, height = im.size
        cnt_small += (width < 32 or height < 32)
        if height == max_height:
            display(im)
        if width == max_width:
            display(im)
        if abs(max(width, height) / min(width, height) - max_ratio) < 0.001:
            display(im)

print(f'number of small images: {cnt_small}')

FileNotFoundError: [Errno 2] No such file or directory: 'photos/male'

<h1>Model Training</h1>

In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

In [8]:
im_size = 32

In [9]:
transform = transforms.Compose([
    #transforms.ToPILImage(),
    # images have different sizes, so we need to resize them
    transforms.Resize((im_size, im_size), interpolation=2),
    transforms.ToTensor(),
    # transform PIL image range [0, 1] to [-1, 1] (this helps in faster convergence)
    transforms.Normalize((.5, .5, .5), (.5, .5, .5))])                                 

In [10]:
train_str = 'train'
test_str = 'test'

def load_dataset(name):
    data_path = f'../input/male-female-faces/data/{name}/'
    dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=transform
    )
    print(dataset)
    loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=64,
        num_workers=0,
        shuffle=True
    )
    return loader

In [11]:
trainloader = load_dataset(train_str)
testloader = load_dataset(test_str)

Dataset ImageFolder
    Number of datapoints: 90000
    Root location: ../input/male-female-faces/data/train/
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )
Dataset ImageFolder
    Number of datapoints: 10002
    Root location: ../input/male-female-faces/data/test/
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )


In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)  # to calculate max of input patch values
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # fully connected layer
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, len(categories))

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)  # make matrix (from tensor) having 16*5*5 columns (= numpy reshape)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [14]:
for epoch in range(9):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

Finished Training


In [15]:
PATH = './model.pth'
torch.save(net.state_dict(), PATH)

In [16]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 500 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 500 test images: 94 %
